In [4]:
import numpy as np
max_pad_len = 216
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs



In [5]:
# Load various imports 
import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset 
fulldatasetpath = '../ESC-50/audio/'

metadata = pd.read_csv('../ESC-50/meta/esc50.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),str(row["filename"]))
    class_label = row["category"]
    print(class_label)
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files')

or_wood_knock
clapping
clapping
clapping
dog
clapping
thunderstorm
fireworks
fireworks
fireworks
fireworks
clapping
clapping
clapping
chainsaw
airplane
mouse_click
pouring_water
train
sheep
water_drops
water_drops
water_drops
church_bells
church_bells
clock_alarm
keyboard_typing
wind
clock_alarm
footsteps
footsteps
footsteps
footsteps
footsteps
footsteps
frog
frog
fireworks
fireworks
cow
water_drops
brushing_teeth
brushing_teeth
car_horn
crackling_fire
helicopter
helicopter
helicopter
helicopter
helicopter
helicopter
drinking_sipping
rain
crackling_fire
insects
crackling_fire
crackling_fire
crackling_fire
laughing
frog
hen
hen
helicopter
helicopter
engine
engine
breathing
mouse_click
crying_baby
frog
frog
frog
hand_saw
car_horn
coughing
coughing
insects
sheep
sheep
vacuum_cleaner
vacuum_cleaner
vacuum_cleaner
chainsaw
chainsaw
chainsaw
glass_breaking
cow
snoring
toilet_flush
pig
pig
pig
pig
pig
crying_baby
crying_baby
crying_baby
rain
church_bells
washing_machine
clock_tick
clock_tick


In [18]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [19]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_rows = 40
num_columns = 216
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

In [20]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [21]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 39, 215, 16)       80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 107, 16)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 19, 107, 16)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 106, 32)       2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 53, 32)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 9, 53, 32)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 52, 64)        

In [27]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

#num_epochs = 12
#num_batch_size = 128

num_epochs = 100
num_batch_size = 256

checkpointer = ModelCheckpoint(filepath='saveModels/weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

0 [==============================] - 0s 73us/step - loss: 0.1154 - accuracy: 0.9625 - val_loss: 3.1885 - val_accuracy: 0.5525

Epoch 00003: val_loss improved from 3.28279 to 3.18850, saving model to saveModels/weights.best.basic_cnn.hdf5
Epoch 4/100
1600/1600 [==============================] - 0s 73us/step - loss: 0.0960 - accuracy: 0.9700 - val_loss: 3.2263 - val_accuracy: 0.5400

Epoch 00004: val_loss did not improve from 3.18850
Epoch 5/100
1600/1600 [==============================] - 0s 73us/step - loss: 0.1265 - accuracy: 0.9519 - val_loss: 3.1728 - val_accuracy: 0.5450

Epoch 00005: val_loss improved from 3.18850 to 3.17283, saving model to saveModels/weights.best.basic_cnn.hdf5
Epoch 6/100
1600/1600 [==============================] - 0s 73us/step - loss: 0.1153 - accuracy: 0.9600 - val_loss: 3.1613 - val_accuracy: 0.5525

Epoch 00006: val_loss improved from 3.17283 to 3.16130, saving model to saveModels/weights.best.basic_cnn.hdf5
Epoch 7/100
1600/1600 [=========================

In [29]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.996874988079071
Testing Accuracy:  0.5950000286102295
